In [ ]:
import pandas as pd

前回加工したデータを読み込む。

In [ ]:
train = pd.read_csv('./processed/train1.csv', index_col=0)
test = pd.read_csv('./processed/test1.csv', index_col=0)

In [ ]:
train.head(3)

In [ ]:
test.head(3)

## 2.1.ダミー変数

　Sex、EmbarkedやTitleなどは文字や言葉で表されるデータであり、質的データという。これら質的データを0と1のダミー変数で表現することで、定量的な分析に取り入れる。<br>
　get_dummiesメソッドを用いることで、質的データをダミー変数に変換する。

In [ ]:
datasets = [train, test]
dummies = []
columns = ['Sex', 'Embarked', 'Title']

for dataset in datasets:
    dummies.append(pd.get_dummies(dataset[columns]))

In [ ]:
dummies[0].head(3)#train

In [ ]:
dummies[1].head(3)#test

　作成したダミー変数のDataFrameをtrainとtestに追加する。

In [ ]:
train = pd.concat([train, dummies[0]], axis=1)
test = pd.concat([test, dummies[1]], axis=1)

　質的データがダミー変数によって表現されていることを確認しよう。

In [ ]:
train.info()

In [ ]:
test.info()

　これで全てのデータが量的データで表された。

## 2.2.データの視覚化

　それでは各属性とSurvivedの関係を視覚化してみよう。<br>
　データの視覚化にはmatplotlibとseabornというライブラリを用いる。

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
sns.set_palette('muted')

　FacetGridクラスは、pandasのDataFrameとmatplotlibを関連づけるAPIである。

In [ ]:
sns.FacetGrid(train, hue='Survived', aspect=4 )

　mapメソッドを用いて、プロット関数を適用しグラフを描画する。今回はsns.kdeplot(カーネル密度関数)を適用した。カーネル密度推定を行うことで、確率密度関数が描画できる。<br>
　setメソッドを用いて、グラフの見た目を調整する。今回は$x$軸の範囲を定めた。<br>
　add_legendメソッドを用いて、グラフに凡例を追加する。

In [ ]:
age_kde = sns.FacetGrid(train, hue='Survived', aspect=4 )
age_kde.map(sns.kdeplot, 'Age', shade=True )
age_kde.set(xlim=(0 , train['Age'].max()))
age_kde.add_legend()

In [ ]:
fare_kde = sns.FacetGrid(train, hue='Survived', aspect=4)
fare_kde.map(sns.kdeplot, 'Fare', shade=True )
fare_kde.set(xlim=(0 , train['Fare'].max()))
fare_kde.add_legend()

　subplots関数を用いることで、ひとつのfigure(図)に複数のaxis(座標軸)を描画することができる。<br>
　barplotメソッドを用いることで、エラーバー付き棒グラフを描画することができる。

In [ ]:
fig, axis = plt.subplots(1,5,figsize=(20, 6))

sns.barplot(x = 'Pclass', y = 'Survived', order=[1,2,3], data=train, ax = axis[0])
sns.barplot(x = 'Sex', y = 'Survived', data=train, ax = axis[2])
sns.barplot(x = 'Embarked', y = 'Survived', data=train, ax = axis[1])
sns.barplot(x = 'IsAlone', y = 'Survived', order=[1,0], data=train, ax = axis[3])
sns.barplot(x = 'Title', y = 'Survived', data=train, ax = axis[4])

　以上の視覚化から以下のようなことがわかる。
 
- Age

> - 10代前半までは**死亡率**よりも**生存率**の方が高い。


> - 10代後半から30代前半までは**生存率**よりも**死亡率**の方が高い。


> - 50代後半からは**生存率**よりも**死亡率**の方が高い。
 
- Fare

> - パレート分布に従う。


> - 低い価格帯で顕著に**生存率**が低い。
 
- Pclass

> グレードの高い客室ほど**生存率**が高い。

- Embarked

> Cがもっとも**生存率**が高い。

- Sex

> 女性の方が**生存率**が高い。

- IsAlone

> 家族がいた方が**生存率**が高い。

- Title

> - Master、Otherの標準誤差が他に比べて大きい。


> - Mrがもっとも**生存率**が低く、Mrs、Miss、Master、Otherの順で**生存率**が低くなる。

　客室のグレード、性別ごとの生存者および死亡者数のヒストグラムを見てみよう。

In [ ]:
sns.barplot(x='Sex', y='Survived', data=train, hue='Pclass')

　さらに詳しく、年齢の分布を付け加えて描画する。

In [ ]:
h = sns.FacetGrid(train, row = 'Sex', col = 'Pclass', hue = 'Survived')
h.map(plt.hist, 'Age', alpha = .85)
h.add_legend()

　三等客室、男性とくに20代から30代の死亡者が顕著に多い。反対に、一等客室の女性はほとんど全員が生き残ったことがわかる。

 最後に、加工したデータをCSVファイルで保存しておく。

In [ ]:
train.to_csv('./processed/train2.csv')
test.to_csv('./processed/test2.csv')